In [11]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import pprint
import env
from time import sleep
from random import randint

pp = pprint.PrettyPrinter(indent=4)

import re
# from selenium.webdriver.chrome.service import Service

# from selenium.webdriver.support.ui import WebDriverWait

# from selenium.webdriver.support import expected_conditions as EC


import codecs


import hmac
import json 



headers = {"Authorization": f"token {env.github_token}", f"User-Agent": f'{env.github_username}'}

In [5]:
##Project stuff

In [6]:
#This ends up taking the top 150 results from github from three independent languages where the readme is written in english and the license is MIT 
# the last part was just a heuristic to help ensure that the results are as professional as possible and hopefully have a read.me
# the sleep was make the query look more like a human clicking through it ( although it was not that sophisticated)
# the code is also implemented so it loops through each of the urls before switching the page number, again this was to create the illusion of seperate searches 


urls=[]
url1=[]
url2=[]
url3=[]
# Here we generate all our urls then we zip them into a tuple
for i in range(1,51):  
    url1.append(f"https://github.com/search?l=&p={str(i)}&q=language%3AC+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en")
    url2.append(f'https://github.com/search?l=&p={str(i)}&q=language%3APython+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en')
    url3.append(f"https://github.com/search?l=&p={str(i)}&q=language%3AJava+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en")

tempurls=list(zip(url1,url2,url3))



In [ ]:
for u in tempurls:
        sleep(randint(3,5))
        for ui in u:

            soup=BeautifulSoup(get(url=ui,headers=headers).content,'html.parser')
            # print(i) 
            # print(u)
            # # Uncomment above if you need to see the urls as they are being populated
            data=soup.find_all('a',class_='v-align-middle') 
            for d in data:
                urls.append(d.string)
              

In [ ]:

# newurls=[]
# for i in urllistactual:
#     i=i.replace('//','')
#     i=re.sub(r'((network))|((members))|((stargazers))','', i,  count=0, flags=0)
#     i=i=re.sub(r'((//))','/', i,  count=0, flags=0)
#     newurls.append(i)
  

len(urls)

270

In [ ]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = urls

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)



